In [ ]:
# Copyright (c) 2024, Edward Jakunskas, Department of Electronic Engineering, Maynooth University
#
# Maintainer: Edward Jakunskas (edward.jakunskas@mu.ie)
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice, this
#    list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


In [1]:
import os
import cv2
import numpy as np
import csv
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Function to calculate the RGB metric and ignore black pixels
def calculate_ngrdi(image):
    # Split the image into its red, green, and blue components
    blue, green, red = cv2.split(image.astype(float) + 1e-6)  # Add small value to avoid division by zero
    
    ngrdi = (green - red) / (red + green)
    
    # Mask out black pixels (where all channels are < 10)
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    ngrdi[~mask_non_black] = np.nan  # Set black pixels to NaN
    
    return ngrdi, mask_non_black

def calculate_tgi(image):
    blue, green, red = cv2.split(image.astype(float) + 1e-6)
    
    #wavelengths for the broad red, green, and blue bands (approximate values)
    lambda_r = 640  # (nm)
    lambda_g = 550  #  (nm)
    lambda_b = 480  # (nm)

    tgi = -0.5 * ((lambda_r - lambda_b) * (red - green) - (lambda_r - lambda_g) * (red - blue))


    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    tgi[~mask_non_black] = np.nan
    
    return tgi, mask_non_black

def calculate_gli(image):
    blue, green, red = cv2.split(image.astype(float))
    
    # GLI: (2 * Green - Red - Blue) / (2 * Green + Red + Blue)
    gli = (2 * green - red - blue) / (2 * green + red + blue + 1e-6)
    
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    gli[~mask_non_black] = np.nan

    return gli, mask_non_black

def calculate_vari(image):
    blue, green, red = cv2.split(image.astype(float) + 1e-6)
    
    # VARI: (Green - Red) / (Green + Red - Blue)
    vari = (green - red) / (green + red - blue + 1e-6)
    
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    vari[~mask_non_black] = np.nan  # Set black pixels to NaN

    return vari, mask_non_black

def calculate_hi(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    hue_channel = hsv_image[:, :, 0].astype(float)
    
   
    normalized_hue = hue_channel / 180.0 #normalize the hue values to range from 0 to 1 (originally 0-180)
    
    blue, green, red = cv2.split(image.astype(float))
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    normalized_hue[~mask_non_black] = np.nan

    return normalized_hue, mask_non_black

def calculate_a_channel(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    a_channel = lab_image[:, :, 1].astype(float)  # The 'a*' channel is the second channel in Lab
    
    blue, green, red = cv2.split(image.astype(float))
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    a_channel[~mask_non_black] = np.nan 
    
    return a_channel, mask_non_black

def calculate_red_channel(image):
    blue, green, red = cv2.split(image.astype(float) + 1e-6) 
    
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    red[~mask_non_black] = np.nan
    
    return red, mask_non_black

def calculate_green_channel(image):
    blue, green, red = cv2.split(image.astype(float) + 1e-6)
    
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    green[~mask_non_black] = np.nan 
    
    return green, mask_non_black

def calculate_blue_channel(image):
    blue, green, red = cv2.split(image.astype(float) + 1e-6)
    
    mask_non_black = (red > 10) | (green > 10) | (blue > 10)
    blue[~mask_non_black] = np.nan
    
    return blue, mask_non_black


In [2]:

# compute statistics for non-black pixels
def compute_statistics(metric):
    valid_pixels = metric[~np.isnan(metric)]
    
    if len(valid_pixels) > 0:
        mode_result = stats.mode(valid_pixels) if len(valid_pixels) > 0 else None
        mode_val = mode_result[0] if mode_result else None
        mean_val = np.mean(valid_pixels) if len(valid_pixels) > 0 else None
        median_val = np.median(valid_pixels) if len(valid_pixels) > 0 else None
        max_val = np.max(valid_pixels) if len(valid_pixels) > 0 else None
        min_val = np.min(valid_pixels) if len(valid_pixels) > 0 else None
    
        std_dev = np.std(valid_pixels)
        variance = np.var(valid_pixels)
        skewness = stats.skew(valid_pixels)
        kurtosis = stats.kurtosis(valid_pixels)
        range_val = np.ptp(valid_pixels)
        q1 = np.percentile(valid_pixels, 25)
        q3 = np.percentile(valid_pixels, 75)
        iqr = q3 - q1
    
        return [mode_val, mean_val, median_val, max_val, min_val, std_dev, variance, skewness, kurtosis, range_val, q1, q3, iqr]
    return [None] * 13

def normalize_image(metric):
    norm = Normalize(vmin=np.nanmin(metric), vmax=np.nanmax(metric))
    normalized_metric = norm(metric)
    return normalized_metric

def process_metrics(image_path, metrics_dir, histograms_dir, stats_dir, metric_name, metric_func):
    image = cv2.imread(image_path)
    metric, mask_non_black = metric_func(image)
    
    normalized_metric = normalize_image(metric)

    original_image_name = os.path.splitext(os.path.basename(image_path))[0]
    normalized_image_path = os.path.join(metrics_dir, f"{original_image_name}_{metric_name}.png")
    cv2.imwrite(normalized_image_path, (normalized_metric * 255).astype(np.uint8))

    stats = compute_statistics(metric)

    # Histogram data
    valid_metric_values = metric[~np.isnan(metric)].flatten()
    bins = 100
    hist_values, bin_edges = np.histogram(valid_metric_values, bins=bins)

    # Plot and save the histogram
    plt.figure()
    plt.hist(valid_metric_values, bins=bin_edges, color='blue', alpha=0.7)
    plt.title(f"Histogram of {metric_name} for {original_image_name}")
    plt.xlabel(f"{metric_name} values")
    plt.ylabel("Frequency")
    plt.xlim([np.min(bin_edges), np.max(bin_edges)])
    plt.ylim([0, np.max(hist_values) + 10])
    
    histogram_image_path = os.path.join(histograms_dir, f"{original_image_name}_histogram_{metric_name}.png")
    plt.savefig(histogram_image_path, format='png')
    plt.close()

    return stats, (bin_edges, hist_values)

# Main function to process all images in the results folder
def run_metrics_on_folder(results_folder):
    histograms_dir = os.path.join(results_folder, 'histograms')
    metrics_dir = os.path.join(results_folder, 'metrics')
    stats_dir = os.path.join(results_folder, 'csvs')
    os.makedirs(histograms_dir, exist_ok=True)
    os.makedirs(metrics_dir, exist_ok=True)
    os.makedirs(stats_dir, exist_ok=True)

    image_files = [f for f in os.listdir(results_folder + '/images') if f.endswith(('.png', '.jpg', '.jpeg'))]

    metrics = [
        ("NGRDI", calculate_ngrdi),
        ("TGI", calculate_tgi),
        ("GLI", calculate_gli),
        ("VARI", calculate_vari),
        ("HI", calculate_hi),
        ("a_channel", calculate_a_channel),
        ("RED", calculate_red_channel),
        ("GREEN", calculate_green_channel),
        ("BLUE", calculate_blue_channel)
    ]

    #process each image and create its individual CSV files
    for image_file in image_files:
        image_path = os.path.join(results_folder + '/images', image_file)
        original_image_name = os.path.splitext(image_file)[0]


        stats_csv_path = os.path.join(stats_dir, f"{original_image_name}_stats.csv")
        hist_csv_path = os.path.join(stats_dir, f"{original_image_name}_histograms.csv")
        
        #write header for statistics CSV
        with open(stats_csv_path, 'w', newline='') as stats_csvfile:
            stats_writer = csv.writer(stats_csvfile)
            
            header = []
            for metric_name, _ in metrics:
                header += [f"{metric_name} Mode", f"{metric_name} Mean", f"{metric_name} Median", f"{metric_name} Max", 
                           f"{metric_name} Min", f"{metric_name} Std Dev", f"{metric_name} Variance", f"{metric_name} Skewness", 
                           f"{metric_name} Kurtosis", f"{metric_name} Range", f"{metric_name} Q1", f"{metric_name} Q3", f"{metric_name} IQR"]
            stats_writer.writerow(header)  #write all metric headers on a single line
            
            row_data = []
            
            with open(hist_csv_path, 'w', newline='') as hist_csvfile:
                hist_writer = csv.writer(hist_csvfile)

                for metric_name, metric_func in metrics:
                    try:
                        metric_stats, (bin_edges, hist_values) = process_metrics(image_path, metrics_dir, histograms_dir, stats_dir, metric_name, metric_func)
                        
                        row_data += metric_stats

                        hist_writer.writerow([metric_name, "edge"] + list(bin_edges))
                        hist_writer.writerow([metric_name, "value"] + list(hist_values))

                    except Exception as e:
                        print(f"Error processing {metric_name} for {image_file}: {e}")
                        return #if an error is detected something went wrong in previous steps and should be fixed
            
            #write the combined statistics for this image on the second line of the CSV
            stats_writer.writerow(row_data)

    print(f"Processing complete.")



In [ ]:
base_dir = '/workspace/Data'
dates = ['visualisation']
for date in dates:
    results_folder = os.path.join(base_dir, date, date + '_results_rgb')
    run_metrics_on_folder(results_folder)